# Uso de los modelos con nuevos datos

In [1]:
import librosa
import joblib
import numpy as np

In [2]:
# Ejemplo audio real
audio_path_real = 'dataset/validation/real/file4.wav_16k.wav_norm.wav_mono.wav_silence.wav'

# Ejemplo audio fake
audio_path_fake = 'dataset/validation/fake/file1.mp3.wav_16k.wav_norm.wav_mono.wav_silence.wav'

# Array con varios audios reales
audio_path_real_array = [
    'dataset/validation/real/file5.wav_16k.wav_norm.wav_mono.wav_silence.wav',
    'dataset/validation/real/file8.wav_16k.wav_norm.wav_mono.wav_silence.wav',
    'dataset/validation/real/file18.wav_16k.wav_norm.wav_mono.wav_silence.wav',
    'dataset/validation/real/file20.wav_16k.wav_norm.wav_mono.wav_silence.wav',
    'dataset/validation/real/file21.wav_16k.wav_norm.wav_mono.wav_silence.wav',
    'dataset/validation/real/file37.wav_16k.wav_norm.wav_mono.wav_silence.wav',
    'dataset/validation/real/file38.wav_16k.wav_norm.wav_mono.wav_silence.wav',
    'dataset/validation/real/file40.wav_16k.wav_norm.wav_mono.wav_silence.wav',
    'dataset/validation/real/file45.wav_16k.wav_norm.wav_mono.wav_silence.wav',
    'dataset/validation/real/file46.wav_16k.wav_norm.wav_mono.wav_silence.wav',
]

# Array con varios audios fakes

audio_path_fake_array = [
    'dataset/validation/fake/file242.mp3.wav_16k.wav_norm.wav_mono.wav_silence.wav',
    'dataset/validation/fake/file19.mp3.wav_16k.wav_norm.wav_mono.wav_silence.wav',
    'dataset/validation/fake/file39.mp3.wav_16k.wav_norm.wav_mono.wav_silence.wav',
    'dataset/validation/fake/file47.wav_16k.wav_norm.wav_mono.wav_silence.wav',
    'dataset/validation/fake/file48.wav_16k.wav_norm.wav_mono.wav_silence.wav',
    'dataset/validation/fake/file56.mp3.wav_16k.wav_norm.wav_mono.wav_silence.wav',
    'dataset/validation/fake/file74.wav_16k.wav_norm.wav_mono.wav_silence.wav',
    'dataset/validation/fake/file78.mp3.wav_16k.wav_norm.wav_mono.wav_silence.wav',
    'dataset/validation/fake/file90.mp3.wav_16k.wav_norm.wav_mono.wav_silence.wav',
    'dataset/validation/fake/file94.mp3.wav_16k.wav_norm.wav_mono.wav_silence.wav',
]

## Características MFCC

In [3]:
# Función para extraer características MFCC de un archivo de audio
def extract_mfcc(file_path, n_mfcc=13):
    try:
        y, sr = librosa.load(file_path, sr=None)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
        mfcc_mean = np.mean(mfcc.T, axis=0)
        return mfcc_mean
    except Exception as e:
        print(f"Error al procesar {file_path}: {e}")
        return None

In [4]:
# Cargo los modelos entrenados (con mayor precisión) para cada algoritmo
random_forest_mfcc = joblib.load('models/random_forest/random_forest_mfcc_model.pkl')
knn_mfcc = joblib.load('models/KNN/knn_mfcc_model_k1150.pkl')
svm_mfcc = joblib.load('models/SVM/svm_mfcc_model.pkl')
mlp_mfcc = joblib.load('models/MLP/mlp_mfcc_model.pkl')
XGBoost_mfcc = joblib.load('models/XGBoost/xgboost_mfcc_model.pkl')

In [5]:
# Determino con cada modelo la probabilidad de que el audio sea real o falso con las características MFCC

# Prediccion con todos los modelos
def determinar_prediccion_all_models_mfcc(new_audio_path):
    # Extraigo las características MFCC del nuevo audio
    new_audio_mfcc = extract_mfcc(new_audio_path)

    # Predicción con Random Forest
    random_forest_mfcc_prediction = random_forest_mfcc.predict_proba([new_audio_mfcc])
    print(f"Random Forest: {random_forest_mfcc_prediction}")

    # Predicción con KNN
    knn_mfcc_prediction = knn_mfcc.predict_proba([new_audio_mfcc])
    print(f"KNN: {knn_mfcc_prediction}")

    # Predicción con SVM
    svm_mfcc_prediction = svm_mfcc.predict_proba([new_audio_mfcc])
    print(f"SVM: {svm_mfcc_prediction}")

    # Predicción con MLP
    mlp_mfcc_prediction = mlp_mfcc.predict_proba([new_audio_mfcc])
    print(f"MLP: {mlp_mfcc_prediction}")

    # Predicción con XGBoost
    XGBoost_mfcc_prediction = XGBoost_mfcc.predict_proba([new_audio_mfcc])
    print(f"XGBoost: {XGBoost_mfcc_prediction} \n\n")

# Predicción del modelo con más precisión (SVM)
def determinar_prediccion_most_accurate_mfcc(new_audio_path):
    # Extraigo las características MFCC del nuevo audio
    new_audio_mfcc = extract_mfcc(new_audio_path)

    # Predicción con SVM
    svm_mfcc_prediction = svm_mfcc.predict_proba([new_audio_mfcc])

    # Predicción del modelo con más precisión (SVM)
    final_prediction = svm_mfcc_prediction
    print(f"Predicción del modelo con mayor precisión (SVM): {final_prediction}")

    # Si la predicción final es mayor a 0.5, el audio es real, de lo contrario es falso
    prob_real = final_prediction[0][0]
    prob_fake = final_prediction[0][1]

    if prob_real > 0.5 and prob_real > prob_fake:
        print("El audio es real \n\n")
    else:
        print("El audio es falso \n\n")

No he tenido en cuenta los modelos con escala aplicada ya que siempre han sacado menor precisión que los modelos sin escala, es por ello que me limito a los modelos sin escala aplicada.

In [6]:
determinar_prediccion_all_models_mfcc(audio_path_real)
determinar_prediccion_all_models_mfcc(audio_path_fake)

Random Forest: [[0.99 0.01]]
KNN: [[0.99043478 0.00956522]]
SVM: [[0.95667605 0.04332395]]
MLP: [[0.9989062 0.0010938]]
XGBoost: [[0.99893886 0.00106115]] 


Random Forest: [[0.07 0.93]]
KNN: [[0.22173913 0.77826087]]
SVM: [[0.19073585 0.80926415]]
MLP: [[1.6570091e-05 9.9998343e-01]]
XGBoost: [[0.02279651 0.9772035 ]] 




Formato de la salida = [[ probabilidad_real    probabilidad_fake ]]

In [7]:
determinar_prediccion_most_accurate_mfcc(audio_path_real)
determinar_prediccion_most_accurate_mfcc(audio_path_fake)

Predicción del modelo con mayor precisión (SVM): [[0.95667605 0.04332395]]
El audio es real 


Predicción del modelo con mayor precisión (SVM): [[0.19073585 0.80926415]]
El audio es falso 




In [8]:
# Pruebas con el array real
for audio in audio_path_real_array:
    determinar_prediccion_most_accurate_mfcc(audio)

Predicción del modelo con mayor precisión (SVM): [[0.95763393 0.04236607]]
El audio es real 


Predicción del modelo con mayor precisión (SVM): [[9.99913165e-01 8.68348755e-05]]
El audio es real 


Predicción del modelo con mayor precisión (SVM): [[0.7990316 0.2009684]]
El audio es real 


Predicción del modelo con mayor precisión (SVM): [[0.97074432 0.02925568]]
El audio es real 


Predicción del modelo con mayor precisión (SVM): [[0.59157155 0.40842845]]
El audio es real 


Predicción del modelo con mayor precisión (SVM): [[9.99236303e-01 7.63697372e-04]]
El audio es real 


Predicción del modelo con mayor precisión (SVM): [[0.31181652 0.68818348]]
El audio es falso 


Predicción del modelo con mayor precisión (SVM): [[0.43109564 0.56890436]]
El audio es falso 


Predicción del modelo con mayor precisión (SVM): [[0.97566746 0.02433254]]
El audio es real 


Predicción del modelo con mayor precisión (SVM): [[0.99186574 0.00813426]]
El audio es real 




In [51]:
# Pruebas con el array fake
for audio in audio_path_fake_array:
    determinar_prediccion_most_accurate_mfcc(audio)

Predicción del modelo con mayor precisión (SVM): [[0.33251997 0.66748003]]
El audio es falso 


Predicción del modelo con mayor precisión (SVM): [[0.30177645 0.69822355]]
El audio es falso 


Predicción del modelo con mayor precisión (SVM): [[0.38864916 0.61135084]]
El audio es falso 


Predicción del modelo con mayor precisión (SVM): [[8.06177032e-09 9.99999992e-01]]
El audio es falso 


Predicción del modelo con mayor precisión (SVM): [[0.01022918 0.98977082]]
El audio es falso 


Predicción del modelo con mayor precisión (SVM): [[0.22366495 0.77633505]]
El audio es falso 


Predicción del modelo con mayor precisión (SVM): [[0.1095941 0.8904059]]
El audio es falso 


Predicción del modelo con mayor precisión (SVM): [[0.03399102 0.96600898]]
El audio es falso 


Predicción del modelo con mayor precisión (SVM): [[0.09468649 0.90531351]]
El audio es falso 


Predicción del modelo con mayor precisión (SVM): [[2.22245205e-06 9.99997778e-01]]
El audio es falso 




## Características espectrograma

No se puede usar ya que no se han implementado los modelos con espectrogranas ya que pesan considerablemente más que los de MFCC y en pruebas previas no se ha determinado una mejora significativa en la precisión.

El código presentado a continuación funciona con el espectrograma, pero no se ha ejecutado por carga computacional.

In [ ]:
# Función para extraer el espectrograma de un archivo de audio con longitud fija
def extract_spectrogram(file_path, n_mels=128, max_len=1300):
    try:
        y, sr = librosa.load(file_path, sr=None)
        S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
        log_S = librosa.power_to_db(S, ref=np.max)
        
        # Ajustar la longitud del espectrograma
        if log_S.shape[1] < max_len:
            pad_width = max_len - log_S.shape[1]
            log_S = np.pad(log_S, ((0, 0), (0, pad_width)), mode='constant')
        elif log_S.shape[1] > max_len:
            log_S = log_S[:, :max_len]
        
        return log_S.flatten()
    except Exception as e:
        print(f"Error al procesar {file_path}: {e}")
        return None

# Aplicar el escalado y PCA a las características del espectrograma

def scale_pca_spectrogram(spectrogram):
    # Cargar el escalador y PCA entrenado
    scaler = joblib.load('data_preprocessed/scalers_PCA/scaler_spectrogram.pkl')
    pca = joblib.load('data_preprocessed/scalers_PCA/pca_spectrogram.pkl')
    
    # Escalar las características del espectrograma
    scaled_spectrogram = scaler.transform(spectrogram.reshape(1, -1))
    
    # Aplicar PCA
    pca_spectrogram = pca.transform(scaled_spectrogram)
    
    return pca_spectrogram

In [ ]:
# Cargo los modelos entrenados (con mayor precisión) para cada algoritmo
random_forest_spectrogram = joblib.load('models/random_forest/random_forest_spectrogram_model.pkl')
knn_spectrogram = joblib.load('models/KNN/knn_spectrogram_model_k1150.pkl')
svm_spectrogram = joblib.load('models/SVM/svm_spectrogram_model.pkl')
mlp_spectrogram = joblib.load('models/MLP/mlp_spectrogram_model.pkl')
XGBoost_spectrogram = joblib.load('models/XGBoost/xgboost_spectrogram_model.pkl')

In [ ]:
# Determino con cada modelo la probabilidad de que el audio sea real o falso con las características del espectrograma

# Prediccion con todos los modelos
def determinar_prediccion_all_models_spectrogram(new_audio_path):
        
    # Extraigo las características spectrogram del nuevo audio
    new_audio_spectrogram = extract_spectrogram(new_audio_path)

    # Escalo y aplico PCA a las características del espectrograma
    new_audio_spectrogram = scale_pca_spectrogram(new_audio_spectrogram)

    # Predicción con Random Forest
    random_forest_spectrogram_prediction = random_forest_spectrogram.predict_proba([new_audio_spectrogram])
    print(f"Random Forest: {random_forest_spectrogram_prediction}")
    # Predicción con KNN
    knn_spectrogram_prediction = knn_spectrogram.predict_proba([new_audio_spectrogram])
    print(f"KNN: {knn_spectrogram_prediction}")

    # Predicción con SVM
    svm_spectrogram_prediction = svm_spectrogram.predict_proba([new_audio_spectrogram])
    print(f"SVM: {svm_spectrogram_prediction}")

    # Predicción con MLP
    mlp_spectrogram_prediction = mlp_spectrogram.predict_proba([new_audio_spectrogram])
    print(f"MLP: {mlp_spectrogram_prediction}")

    # Predicción con XGBoost
    XGBoost_spectrogram_prediction = XGBoost_spectrogram.predict_proba([new_audio_spectrogram])
    print(f"XGBoost: {XGBoost_spectrogram_prediction}")

# Predicción del modelo con más precisión (XXXX)
def determinar_prediccion_most_accurate_spectrogram(new_audio_path):
    # Extraigo las características spectrogram del nuevo audio
    new_audio_spectrogram = extract_spectrogram(new_audio_path)

    # Escalo y aplico PCA a las características del espectrograma
    new_audio_spectrogram = scale_pca_spectrogram(new_audio_spectrogram)

    # Predicción con SVM
    svm_spectrogram_prediction = svm_spectrogram.predict_proba([new_audio_spectrogram])

    # Predicción del modelo con más precisión (SVM)
    final_prediction = svm_spectrogram_prediction
    print(f"Predicción del modelo con mayor precisión (SVM): {final_prediction}")

    # Si la predicción final es mayor a 0.5, el audio es real, de lo contrario es falso
    prob_real = final_prediction[0][0]
    prob_fake = final_prediction[0][1]

    if prob_real > 0.5 and prob_real > prob_fake:
        print("El audio es real")
    else:
        print("El audio es falso")

In [ ]:
determinar_prediccion_all_models_spectrogram(new_audio_path)

Formato de la salida = [[ probabilidad_real    probabilidad_fake ]]

In [ ]:
determinar_prediccion_most_accurate_spectrogram(new_audio_path)